<a href="https://colab.research.google.com/github/mandie021/big-data-challenge/blob/main/level-1/level_1_officeproducts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
#import postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
#create session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
office_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True)
office_df.show()

In [ ]:
#Remove duplicates and na values
office_df = outdoor_df.dropna()
office_df = outdoor_df.dropDuplicates()
office_df.printSchema()

In [ ]:
# maoke colummns int
office_df = review_id_table.withColumn("customer_id", office_df["customer_id"].cast(IntegerType()))
office_df = review_id_table.withColumn("product_parent", office_df["product_parent"].cast(IntegerType()))
office_df = review_id_table.withColumn("star_rating", office_df["star_rating"].cast(IntegerType()))
office_df = review_id_table.withColumn("helpful_votes", office_df["helpful_votes"].cast(IntegerType()))
office_df = review_id_table.withColumn("total_votes", office_df["total_votes"].cast(IntegerType()))

In [ ]:
# Count the number of records (rows) in the dataset
office_df.count()

In [ ]:
# Transform the dataset to fit the tables in the schema file
#imports and review id table
from pyspark.sql.functions import to_date

review_id_table = office_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

In [ ]:
review_id_table.printSchema()

In [ ]:
# Transform the dataset to fit the tables in the schema file
#imports and products table
products = office_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

In [ ]:
products.printSchema()

In [ ]:
# Transform the dataset to fit the tables in the schema file
#imports and customers table
customers = office_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

In [ ]:
#convert count to int
customers = customers.withColumn("customer_count", customers["customer_count"].cast(IntegerType()))

In [ ]:
customers.printSchema()

In [ ]:
# Transform the dataset to fit the tables in the schema file
#imports and vine table
vine_table = office_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

In [ ]:
vine_table.printSchema()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatahwdb.crkggtvwjikn.us-east-1.rds.amazonaws.com:5432/hw_lvl1_db"
config = {"user":"root", 
          "password": "Static2$", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)